In [9]:
import os
from PIL import Image
import random

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm

In [53]:
def read_file_path(path):
    return [os.path.join(path, img) for img in os.listdir(path) if os.path.splitext(img)[1] == '.png']

def train_test_split_file(files, train_rate=0.8):
    files_shuffled = random.sample(files, len(files))
    num_train = int(len(files_shuffled)*train_rate)
    train_files = files_shuffled[:num_train]
    valid_files = files_shuffled[num_train:]
    return train_files, valid_files

files = read_file_path('./train')
train_files, valid_files = train_test_split_file(files)
print(f'train: {len(train_files)}, valid: {len(valid_files)}')

train: 4204, valid: 1052


In [110]:
class CharacterDataset(data.Dataset):
    def __init__(self, files, classes, phase='train'):
        self.files = files
        self.classes = classes
        self.phase = phase # train or valid or test
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        path = self.files[idx]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.bitwise_not(img)
        img = torch.tensor(img, dtype=torch.float)
        img = torch.unsqueeze(img, 0)
        
        if self.phase =='test':
            label = None
        else:
            file_name = os.path.splitext(os.path.basename(path))[0]
            label = file_name.split('_')[-1]
            label = self.classes.index(label)
            label = torch.tensor(label, dtype=torch.long)
        
        return img, label
    
classes = ['hiragana', 'katakana', 'latinalphabet']
train_dataset = CharacterDataset(
    files = train_files,
    classes = classes,
    phase = 'train'
)
valid_dataset = CharacterDataset(
    files = valid_files,
    classes = classes,
    phase = 'valid'
)

idx = 1000
print(train_dataset.__getitem__(idx)[0].shape)
print(train_dataset.__getitem__(idx)[1])

torch.Size([1, 28, 28])
tensor(0)


In [121]:
batch_size = 64

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = data.DataLoader(valid_dataset, batch_size=32, shuffle=False)

dataloaders_dict = {
    'train': train_dataloader, 
    'valid': valid_dataloader
}

batch_iterator = iter(train_dataloader)
inputs, labels = next(batch_iterator)
print(inputs.size())
print(len(labels))

torch.Size([64, 1, 28, 28])
64


In [159]:
class CharacterModel(nn.Module):
    def __init__(self):
        super(CharacterModel, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(in_features=6272, out_features=128) # エラー
        self.fc2 = nn.Linear(in_features=128, out_features=3)
        
    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool2(x)

        x = x.view(-1, 6272)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        
        return x
    
model = CharacterModel()
print(model)

CharacterModel(
  (conv1_1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6272, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=3, bias=True)
)


# Train

In [158]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 30

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-------------')
    
    for phase in ['train', 'valid']:
        
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        epoch_loss = 0.0
        epoch_corrects = 0
        
        for inputs, labels in tqdm(dataloaders_dict[phase]):
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)
                
        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

Epoch 1/1
-------------


<ipython-input-158-32b8e2e9ab29>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(dataloaders_dict[phase]):


tensor([0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        2, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0, 1,
        1, 2, 2, 2, 1, 0, 2, 1, 2, 2, 0, 0, 0, 0, 2, 0])
tensor([1, 1, 1, 0, 2, 2, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 0, 2, 1, 1, 0, 1, 0, 2,
        1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 0, 1, 0, 0, 2, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 2, 0, 2, 0, 2, 2, 1, 0, 2, 2, 1, 1, 0, 0, 0])
tensor([1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 0, 1, 2, 1, 2, 2, 0, 2, 0, 2, 1, 1, 1, 1,
        2, 1, 2, 2, 2, 1, 0, 0, 0, 0, 1, 2, 0, 2, 0, 1, 0, 1, 1, 1, 0, 1, 2, 1,
        0, 1, 1, 2, 1, 0, 0, 0, 1, 1, 1, 1, 2, 0, 2, 1])
tensor([0, 1, 1, 1, 0, 2, 0, 0, 0, 2, 0, 0, 1, 2, 1, 2, 1, 0, 0, 2, 0, 1, 0, 1,
        1, 2, 1, 0, 1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 0, 1, 1, 0, 2, 1, 2, 2, 0, 1,
        2, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0])
tensor([0, 0, 0, 0, 1, 2, 1, 2, 2, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 1, 0, 2, 0, 0,
        1, 2, 0, 2, 0, 2, 2, 0, 2, 1, 1, 1, 1, 0, 1,

tensor([0, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0, 2, 0, 0, 2, 1, 2, 1, 2, 1, 0, 1, 2, 0,
        2, 0, 0, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 2, 2, 2, 0, 2, 1, 1, 1, 0, 1, 2,
        1, 2, 2, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 0, 0, 1])
tensor([2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 1, 2, 2, 0, 1, 1, 2, 0, 1, 2, 2, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 0, 2, 1, 0, 0, 2, 1, 1, 1, 0, 2, 2,
        1, 1, 1, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 1, 1])
tensor([2, 2, 2, 1, 2, 0, 1, 2, 2, 0, 0, 1, 2, 2, 2, 0, 1, 0, 1, 1, 1, 2, 2, 2,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 2, 0, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 2,
        2, 2, 1, 2, 2, 0, 0, 0, 2, 0, 2, 2, 1, 0, 0, 1])
tensor([0, 2, 0, 1, 0, 2, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 2, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 0, 2, 2, 2, 0, 1, 1, 2, 0, 0, 1, 1,
        0, 1, 2, 1, 0, 2, 2, 1, 0, 2, 1, 1, 0, 1, 1, 1])
tensor([1, 1, 0, 1, 1, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 1, 1, 2, 2, 1, 2,
        1, 1, 0, 1, 0, 2, 1, 0, 0, 2, 1, 1, 2, 2, 0,

tensor([2, 0, 1, 0, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 1, 2, 0, 1, 1, 0, 0, 1, 0, 1,
        1, 1, 2, 1, 0, 0, 1, 2])
tensor([1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 1, 0, 2, 0, 0, 2, 1, 1, 2, 1, 2, 0, 0,
        1, 1, 0, 2, 0, 2, 0, 1])
tensor([1, 0, 2, 0, 1, 0, 2, 2, 1, 1, 2, 1, 0, 0, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1,
        1, 0, 2, 1, 2, 2, 2, 0])
tensor([1, 0, 0, 0, 2, 2, 1, 2, 2, 0, 0, 0, 1, 0, 0, 2, 2, 1, 0, 1, 1, 0, 0, 2,
        2, 0, 0, 0, 1, 2, 1, 0])
tensor([1, 0, 0, 2, 2, 1, 2, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 2, 2,
        1, 1, 2, 0, 0, 1, 0, 1])
tensor([1, 1, 1, 0, 1, 1, 0, 2, 2, 2, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 2, 1, 2, 1,
        0, 0, 2, 1, 1, 0, 0, 1])
tensor([0, 2, 0, 1, 2, 1, 0, 2, 0, 1, 1, 1, 2, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
        0, 2, 0, 2, 0, 0, 1, 0])
tensor([1, 0, 1, 2, 0, 0, 0, 2, 1, 1, 1, 0, 0, 1, 0, 2, 0, 2, 1, 0, 0, 1, 0, 0,
        1, 1, 0, 2, 0, 0, 0, 0])
tensor([2, 1, 1, 1, 1, 0, 0, 2, 2, 0, 1, 2, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0,
        1, 1, 0,

In [160]:
model_path = './model.pt'
torch.save(model.state_dict(), './model.pt')
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [192]:
incorrect_files = []
for path in valid_files:
    file = os.path.splitext(os.path.basename(path))[0]
    label = file.split('_')[-1]

    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.bitwise_not(img)
    img = torch.tensor(img, dtype=torch.float)
    img = torch.unsqueeze(img, 0)
    img = torch.unsqueeze(img, 0)
    preds = model(img)

    _, pred = torch.max(preds, 1)
    if label!=classes[pred]:
        print(file, classes[pred])
        incorrect_files.append(path)
        

2607_katakana hiragana
4465_katakana latinalphabet
325_hiragana katakana
1267_katakana hiragana
1181_katakana latinalphabet
1312_katakana hiragana
820_hiragana latinalphabet
4976_hiragana katakana
4496_katakana latinalphabet
4515_katakana hiragana
4128_hiragana katakana
1638_hiragana katakana
3640_hiragana katakana
4828_hiragana katakana
3837_katakana hiragana
555_katakana hiragana
2194_hiragana latinalphabet
3988_latinalphabet hiragana
5233_katakana hiragana
4821_hiragana latinalphabet
3329_latinalphabet hiragana
2994_hiragana katakana
982_hiragana katakana
377_hiragana katakana
4089_latinalphabet katakana
1372_latinalphabet katakana
3439_latinalphabet hiragana
3200_katakana hiragana
4107_hiragana latinalphabet
610_katakana hiragana
231_hiragana katakana
2529_katakana hiragana
2373_hiragana katakana
4110_hiragana katakana
1868_katakana hiragana
2517_katakana latinalphabet
3859_katakana hiragana
3187_katakana hiragana
1493_hiragana katakana
4516_katakana hiragana
2987_hiragana latinalp

In [206]:
from PIL import Image

Image.open(incorrect_files[0])